In [11]:
import pandas as pd
import numpy as np
import requests
import json
import time
from datetime import datetime
import os

os.makedirs('../data/raw', exist_ok=True)
os.makedirs('../data/processed', exist_ok=True)

print("Setup complete!")

Setup complete!


In [10]:
import sys
print(sys.executable)

/opt/anaconda3/bin/python


In [3]:
!pip install sec-api


[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import sys
!{sys.executable} -m pip install sec-api

  Using cached sec_api-1.0.32-py3-none-any.whl.metadata (66 kB)
Using cached sec_api-1.0.32-py3-none-any.whl (24 kB)


In [5]:
SEC_API_KEY = '8b7992997602c91a2e70cf514af3e72d7b9418219f2e2362a9a75e758f378dd2'  

from sec_api import QueryApi

queryApi = QueryApi(api_key=SEC_API_KEY)

print("API connection successful!")

API connection successful!


In [6]:
# Sample companies (ticker symbols)
# Starting with well-known tech/software companies
companies = {
    'AAPL': 'Apple Inc.',
    'MSFT': 'Microsoft Corporation',
    'GOOGL': 'Alphabet Inc.',
    'META': 'Meta Platforms Inc.',
    'NVDA': 'NVIDIA Corporation',
    'CRM': 'Salesforce Inc.',
    'ADBE': 'Adobe Inc.',
    'ORCL': 'Oracle Corporation',
    'NOW': 'ServiceNow Inc.',
    'SNOW': 'Snowflake Inc.'
}

print(f"Selected {len(companies)} companies for analysis")
print(companies)

Selected 10 companies for analysis
{'AAPL': 'Apple Inc.', 'MSFT': 'Microsoft Corporation', 'GOOGL': 'Alphabet Inc.', 'META': 'Meta Platforms Inc.', 'NVDA': 'NVIDIA Corporation', 'CRM': 'Salesforce Inc.', 'ADBE': 'Adobe Inc.', 'ORCL': 'Oracle Corporation', 'NOW': 'ServiceNow Inc.', 'SNOW': 'Snowflake Inc.'}


In [7]:
def get_company_filings(ticker, form_type='10-K', max_filings=3):
    """
    Get recent SEC filings for a company
    
    Parameters:
    - ticker: Stock ticker symbol
    - form_type: Type of filing (10-K for annual, 10-Q for quarterly)
    - max_filings: Number of recent filings to retrieve
    """
    
    query = {
        "query": f'ticker:{ticker} AND formType:"{form_type}"',
        "from": "0",
        "size": str(max_filings),
        "sort": [{"filedAt": {"order": "desc"}}]
    }
    
    try:
        response = queryApi.get_filings(query)
        filings = response['filings']
        
        print(f"Found {len(filings)} {form_type} filings for {ticker}")
        
        return filings
    
    except Exception as e:
        print(f"Error fetching filings for {ticker}: {e}")
        return []

test_filings = get_company_filings('AAPL', max_filings=3)
print(f"\nExample filing data for AAPL:")
if test_filings:
    print(f"Most recent filing date: {test_filings[0]['filedAt']}")
    print(f"Fiscal year: {test_filings[0].get('fiscalYear', 'N/A')}")

Found 3 10-K filings for AAPL

Example filing data for AAPL:
Most recent filing date: 2025-10-31T06:01:26-04:00
Fiscal year: N/A


In [12]:
all_filings = {}

for ticker, company_name in companies.items():
    print(f"\nFetching filings for {ticker} ({company_name})...")
    filings = get_company_filings(ticker, form_type='10-K', max_filings=3)
    all_filings[ticker] = filings
    
    time.sleep(1)

print(f"\n{'='*50}")
print(f"Data collection complete!")
print(f"Total companies: {len(all_filings)}")
print(f"Total filings collected: {sum(len(f) for f in all_filings.values())}")


Fetching filings for AAPL (Apple Inc.)...
Found 3 10-K filings for AAPL

Fetching filings for MSFT (Microsoft Corporation)...
Found 3 10-K filings for MSFT

Fetching filings for GOOGL (Alphabet Inc.)...
Found 1 10-K filings for GOOGL

Fetching filings for META (Meta Platforms Inc.)...
Found 3 10-K filings for META

Fetching filings for NVDA (NVIDIA Corporation)...
Found 3 10-K filings for NVDA

Fetching filings for CRM (Salesforce Inc.)...
Found 3 10-K filings for CRM

Fetching filings for ADBE (Adobe Inc.)...
Found 3 10-K filings for ADBE

Fetching filings for ORCL (Oracle Corporation)...
Found 3 10-K filings for ORCL

Fetching filings for NOW (ServiceNow Inc.)...
Found 3 10-K filings for NOW

Fetching filings for SNOW (Snowflake Inc.)...
Found 3 10-K filings for SNOW

Data collection complete!
Total companies: 10
Total filings collected: 28


In [13]:
import json

output_file = '../data/raw/company_filings.json'

with open(output_file, 'w') as f:
    json.dump(all_filings, f, indent=2)

print(f"Raw filings data saved to {output_file}")
print(f"File size: {os.path.getsize(output_file) / 1024:.2f} KB")

Raw filings data saved to ../data/raw/company_filings.json
File size: 193.09 KB


In [14]:
sample_filing = all_filings['AAPL'][0]

print("Available keys in a filing:")
print(list(sample_filing.keys()))
print("\n" + "="*50)

if 'financials' in sample_filing:
    print("\nFinancial data available!")
    print(sample_filing['financials'].keys() if sample_filing['financials'] else "No financial details")
else:
    print("\nNo structured financial data in basic filing response")
    print("We'll need to parse the actual filing documents")

Available keys in a filing:
['ticker', 'formType', 'accessionNo', 'cik', 'companyNameLong', 'companyName', 'linkToFilingDetails', 'description', 'linkToTxt', 'filedAt', 'documentFormatFiles', 'periodOfReport', 'entities', 'id', 'seriesAndClassesContractsInformation', 'linkToHtml', 'linkToXbrl', 'dataFiles']


No structured financial data in basic filing response
We'll need to parse the actual filing documents


In [15]:
from sec_api import ExtractorApi

extractorApi = ExtractorApi(SEC_API_KEY)

test_url = all_filings['AAPL'][0]['linkToFilingDetails']
print(f"Extracting data from: {test_url}")

try:
    revenues = extractorApi.get_section(test_url, "1A", "text")
    print("\nSuccessfully connected to Extractor API")
    
except Exception as e:
    print(f"Error: {e}")

Extracting data from: https://www.sec.gov/Archives/edgar/data/320193/000032019325000079/aapl-20250927.htm

Successfully connected to Extractor API


In [16]:
def extract_financial_metrics(filing_url, ticker):
    """
    Extract key financial metrics from a 10-K filing
    Uses XBRL data extraction
    """
    
    try:
        
        metrics = {
            'ticker': ticker,
            'filing_url': filing_url
        }
        
        xbrl_metrics = [
            'Revenues',
            'RevenueFromContractWithCustomerExcludingAssessedTax',
            'OperatingIncomeLoss',
            'NetIncomeLoss', 
            'EarningsPerShareBasic',
            'Assets',
            'Liabilities',
            'StockholdersEquity',
            'CashAndCashEquivalentsAtCarryingValue',
            'OperatingExpenses'
        ]
        
        print(f"Extracting metrics for {ticker}...")
        
    
        metrics['extracted'] = True
        
        return metrics
        
    except Exception as e:
        print(f"Error extracting data for {ticker}: {e}")
        return {'ticker': ticker, 'error': str(e)}

test_metrics = extract_financial_metrics(all_filings['AAPL'][0]['linkToFilingDetails'], 'AAPL')
print(test_metrics)

Extracting metrics for AAPL...
{'ticker': 'AAPL', 'filing_url': 'https://www.sec.gov/Archives/edgar/data/320193/000032019325000079/aapl-20250927.htm', 'extracted': True}


In [17]:
from sec_api import XbrlApi

xbrlApi = XbrlApi(SEC_API_KEY)

def get_financial_data(accession_no, ticker):
    """
    Extract financial data using XBRL API
    accession_no: The SEC accession number for the filing
    """
    
    try:
        xbrl_json = xbrlApi.xbrl_to_json(accession_no)
        
        metrics = {
            'ticker': ticker,
            'accession_no': accession_no
        }
        
        
        if 'BalanceSheets' in xbrl_json:
            balance_sheet = xbrl_json['BalanceSheets']
            print(f"✓ Found Balance Sheet data for {ticker}")
            
        if 'StatementsOfIncome' in xbrl_json:
            income_stmt = xbrl_json['StatementsOfIncome']
            print(f"✓ Found Income Statement data for {ticker}")
            
        metrics['has_data'] = True
        return xbrl_json
        
    except Exception as e:
        print(f"✗ Error for {ticker}: {e}")
        return None

apple_accession = all_filings['AAPL'][0]['accessionNo']
print(f"Testing with Apple accession number: {apple_accession}\n")

apple_data = get_financial_data(apple_accession, 'AAPL')

if apple_data:
    print(f"\nAvailable sections in XBRL data:")
    print(list(apple_data.keys()))

Testing with Apple accession number: 0000320193-25-000079

✗ Error for AAPL: API error: 400 - {"status":400,"error":"request failed"}


In [19]:
import sys
!{sys.executable} -m pip install yfinance

  Using cached yfinance-1.1.0-py2.py3-none-any.whl.metadata (6.1 kB)
  Using cached multitasking-0.0.12-py3-none-any.whl
  Using cached peewee-3.19.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached curl_cffi-0.13.0-cp39-abi3-macosx_11_0_arm64.whl.metadata (13 kB)
  Using cached websockets-16.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.8 kB)
Using cached yfinance-1.1.0-py2.py3-none-any.whl (129 kB)
Using cached curl_cffi-0.13.0-cp39-abi3-macosx_11_0_arm64.whl (3.0 MB)
Using cached peewee-3.19.0-py3-none-any.whl (411 kB)
Using cached websockets-16.0-cp312-cp312-macosx_11_0_arm64.whl (175 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [yfinance]3/5 [curl_cffi]


In [20]:
import yfinance as yf

def get_company_financials(ticker):
    """
    Get financial data using Yahoo Finance
    This pulls from SEC filings but in a cleaner format
    """
    
    try:
        stock = yf.Ticker(ticker)
        
        financials = stock.financials  # Income statement (annual)
        balance_sheet = stock.balance_sheet  # Balance sheet
        cashflow = stock.cashflow  # Cash flow statement
        
        info = stock.info
        
        print(f"✓ Successfully retrieved data for {ticker}")
        
        return {
            'ticker': ticker,
            'company_name': info.get('longName', 'N/A'),
            'sector': info.get('sector', 'N/A'),
            'industry': info.get('industry', 'N/A'),
            'market_cap': info.get('marketCap', None),
            'financials': financials,
            'balance_sheet': balance_sheet,
            'cashflow': cashflow,
            'info': info
        }
        
    except Exception as e:
        print(f"✗ Error retrieving data for {ticker}: {e}")
        return None

print("Testing with Apple (AAPL):\n")
apple_financials = get_company_financials('AAPL')

if apple_financials:
    print(f"\nCompany: {apple_financials['company_name']}")
    print(f"Sector: {apple_financials['sector']}")
    print(f"Market Cap: ${apple_financials['market_cap']:,}")
    print(f"\nFinancial statement columns (years): {list(apple_financials['financials'].columns)}")
    print(f"\nAvailable metrics: {len(apple_financials['financials'])} income statement items")

Testing with Apple (AAPL):

✓ Successfully retrieved data for AAPL

Company: Apple Inc.
Sector: Technology
Market Cap: $4,060,008,153,088

Financial statement columns (years): [Timestamp('2025-09-30 00:00:00'), Timestamp('2024-09-30 00:00:00'), Timestamp('2023-09-30 00:00:00'), Timestamp('2022-09-30 00:00:00'), Timestamp('2021-09-30 00:00:00')]

Available metrics: 39 income statement items


In [21]:
all_financial_data = {}

for ticker in companies.keys():
    print(f"\nFetching financial data for {ticker}...")
    company_data = get_company_financials(ticker)
    
    if company_data:
        all_financial_data[ticker] = company_data
    
    time.sleep(0.5)

print(f"\n{'='*60}")
print(f"Financial data collection complete!")
print(f"Successfully collected data for {len(all_financial_data)} companies")


Fetching financial data for AAPL...
✓ Successfully retrieved data for AAPL

Fetching financial data for MSFT...
✓ Successfully retrieved data for MSFT

Fetching financial data for GOOGL...
✓ Successfully retrieved data for GOOGL

Fetching financial data for META...
✓ Successfully retrieved data for META

Fetching financial data for NVDA...
✓ Successfully retrieved data for NVDA

Fetching financial data for CRM...
✓ Successfully retrieved data for CRM

Fetching financial data for ADBE...
✓ Successfully retrieved data for ADBE

Fetching financial data for ORCL...
✓ Successfully retrieved data for ORCL

Fetching financial data for NOW...
✓ Successfully retrieved data for NOW

Fetching financial data for SNOW...
✓ Successfully retrieved data for SNOW

Financial data collection complete!
Successfully collected data for 10 companies


In [22]:
def calculate_pe_metrics(ticker, financial_data):
    """
    Calculate key PE metrics from financial statements
    """
    
    try:
        financials = financial_data['financials']
        balance_sheet = financial_data['balance_sheet']
        cashflow = financial_data['cashflow']
        info = financial_data['info']
        
        latest_year = financials.columns[0]
        
        metrics = {
            'ticker': ticker,
            'company_name': financial_data['company_name'],
            'sector': financial_data['sector'],
            'industry': financial_data['industry'],
            'year': latest_year.year,
        }
        
        if 'Total Revenue' in financials.index:
            revenue = financials.loc['Total Revenue', latest_year]
            metrics['revenue'] = revenue
            
            if len(financials.columns) > 1:
                prior_revenue = financials.loc['Total Revenue', financials.columns[1]]
                metrics['revenue_growth'] = ((revenue - prior_revenue) / prior_revenue) * 100
        
        if 'EBITDA' in financials.index:
            ebitda = financials.loc['EBITDA', latest_year]
            metrics['ebitda'] = ebitda
            if 'revenue' in metrics and metrics['revenue'] != 0:
                metrics['ebitda_margin'] = (ebitda / metrics['revenue']) * 100
        
        if 'Net Income' in financials.index:
            net_income = financials.loc['Net Income', latest_year]
            metrics['net_income'] = net_income
            if 'revenue' in metrics and metrics['revenue'] != 0:
                metrics['net_margin'] = (net_income / metrics['revenue']) * 100
        
        if 'Operating Income' in financials.index:
            operating_income = financials.loc['Operating Income', latest_year]
            metrics['operating_income'] = operating_income
            if 'revenue' in metrics and metrics['revenue'] != 0:
                metrics['operating_margin'] = (operating_income / metrics['revenue']) * 100
        
        if 'Total Assets' in balance_sheet.index:
            metrics['total_assets'] = balance_sheet.loc['Total Assets', latest_year]
        
        if 'Total Debt' in balance_sheet.index:
            total_debt = balance_sheet.loc['Total Debt', latest_year]
            metrics['total_debt'] = total_debt
            
            if 'ebitda' in metrics and metrics['ebitda'] != 0:
                metrics['debt_to_ebitda'] = total_debt / metrics['ebitda']
        
        if 'Free Cash Flow' in cashflow.index:
            fcf = cashflow.loc['Free Cash Flow', latest_year]
            metrics['free_cash_flow'] = fcf
            if 'revenue' in metrics and metrics['revenue'] != 0:
                metrics['fcf_margin'] = (fcf / metrics['revenue']) * 100
        
        metrics['market_cap'] = info.get('marketCap')
        metrics['enterprise_value'] = info.get('enterpriseValue')
        
        if metrics.get('enterprise_value') and metrics.get('revenue'):
            metrics['ev_revenue'] = metrics['enterprise_value'] / metrics['revenue']
        
        if metrics.get('enterprise_value') and metrics.get('ebitda') and metrics['ebitda'] > 0:
            metrics['ev_ebitda'] = metrics['enterprise_value'] / metrics['ebitda']
        
        return metrics
        
    except Exception as e:
        print(f"Error calculating metrics for {ticker}: {e}")
        return None

pe_metrics_list = []

for ticker, data in all_financial_data.items():
    print(f"Calculating PE metrics for {ticker}...")
    metrics = calculate_pe_metrics(ticker, data)
    if metrics:
        pe_metrics_list.append(metrics)

pe_metrics_df = pd.DataFrame(pe_metrics_list)

print(f"\n{'='*60}")
print(f"Successfully calculated metrics for {len(pe_metrics_df)} companies")
print(f"\nMetrics calculated: {list(pe_metrics_df.columns)}")

Calculating PE metrics for AAPL...
Calculating PE metrics for MSFT...
Calculating PE metrics for GOOGL...
Calculating PE metrics for META...
Calculating PE metrics for NVDA...
Calculating PE metrics for CRM...
Calculating PE metrics for ADBE...
Calculating PE metrics for ORCL...
Calculating PE metrics for NOW...
Calculating PE metrics for SNOW...

Successfully calculated metrics for 10 companies

Metrics calculated: ['ticker', 'company_name', 'sector', 'industry', 'year', 'revenue', 'revenue_growth', 'ebitda', 'ebitda_margin', 'net_income', 'net_margin', 'operating_income', 'operating_margin', 'total_assets', 'total_debt', 'debt_to_ebitda', 'free_cash_flow', 'fcf_margin', 'market_cap', 'enterprise_value', 'ev_revenue', 'ev_ebitda']


In [23]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

print("PE Metrics Summary:")
print("="*80)
print(pe_metrics_df[['ticker', 'company_name', 'revenue_growth', 'ebitda_margin', 
                      'operating_margin', 'ev_revenue', 'ev_ebitda']].to_string(index=False))

print("\n" + "="*80)
print("\nFull dataset shape:", pe_metrics_df.shape)
print(f"Columns: {pe_metrics_df.shape[1]}")
print(f"Companies: {pe_metrics_df.shape[0]}")

PE Metrics Summary:
ticker          company_name  revenue_growth  ebitda_margin  operating_margin  ev_revenue  ev_ebitda
  AAPL            Apple Inc.            6.43          34.78             31.97        9.71      27.92
  MSFT Microsoft Corporation           14.93          56.85             45.62       11.01      19.37
 GOOGL         Alphabet Inc.           15.09          44.86             32.03        9.42      21.00
  META  Meta Platforms, Inc.           22.17          52.60             41.44        8.46      16.08
  NVDA    NVIDIA Corporation          114.20          66.01             62.42       34.73      52.61
   CRM      Salesforce, Inc.            8.72          29.40             20.23        4.79      16.29
  ADBE            Adobe Inc.           10.53          41.29             36.63        4.57      11.08
  ORCL    Oracle Corporation            8.38          41.66             31.45        9.96      23.91
   NOW      ServiceNow, Inc.           20.88          22.76            